#### VARIABLES

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import glob
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

In [2]:
basepath = "dades_tfg/data"
demographics = pd.read_csv(os.path.join(basepath, "demographics.csv"))
demographics_N = pd.read_csv(os.path.join(basepath, "demographics_N.csv"))
nodes = pd.read_csv(os.path.join(basepath, "nodes.csv"))
basepath_FA = os.path.join(basepath, "FA")
basepath_GM = os.path.join(basepath, "GM")
basepath_RS = os.path.join(basepath, "RS")

In [3]:
filenames = ["{:04d}.csv".format(x) for x in demographics["id"]]

In [4]:
data_FA = np.zeros(shape=(len(filenames), 76, 76))
data_GM = np.zeros(shape=(len(filenames), 76, 76))
data_RS = np.zeros(shape=(len(filenames), 76, 76))

In [5]:
for i, filename in enumerate(filenames):
    df_FA = pd.read_csv(os.path.join(basepath_FA, filename), header=None)
    data_FA[i,:,:] = df_FA.values
    
    df_GM = pd.read_csv(os.path.join(basepath_GM, filename), header=None)
    data_GM[i,:,:] = df_GM.values
    
    df_RS = pd.read_csv(os.path.join(basepath_RS, filename), header=None)
    data_RS[i,:,:] = df_RS.values

In [6]:
basepath_FA_N = "dades_tfg/data/Naples/DTI_networks"
basepath_GM_N = "dades_tfg/data/Naples/GM_networks"
basepath_RS_N = "dades_tfg/data/Naples/rsfmri_networks"

# Listar nombres de archivos CSV en cada carpeta
filenames_FA_N = sorted([f for f in os.listdir(basepath_FA_N) if f.endswith('.csv')])
filenames_GM_N = sorted([f for f in os.listdir(basepath_GM_N) if f.endswith('.csv')])
filenames_RS_N = sorted([f for f in os.listdir(basepath_RS_N) if f.endswith('.csv')])

In [7]:
# Inicializar arrays para almacenar los datos
data_FA_N = np.zeros((len(filenames_FA_N), 76, 76))  
data_GM_N = np.zeros((len(filenames_GM_N), 76, 76))
data_RS_N = np.zeros((len(filenames_RS_N), 76, 76))

In [8]:
# FA
for i, filename in enumerate(filenames_FA_N):
    df_FA_N = pd.read_csv(os.path.join(basepath_FA_N, filename), header=None)
    data_FA_N[i, :, :] = df_FA_N.values

#  GM
for i, filename in enumerate(filenames_GM_N):
    df_GM_N = pd.read_csv(os.path.join(basepath_GM_N, filename), header=None)
    data_GM_N[i, :, :] = df_GM_N.values

#  RS
for i, filename in enumerate(filenames_RS_N):
    df_RS_N = pd.read_csv(os.path.join(basepath_RS_N, filename), header=None)
    data_RS_N[i, :, :] = df_RS_N.values

In [9]:
num_samples_FA = len(data_FA)
num_samples_FA_N = len(data_FA_N)
num_samples_GM = len(data_FA)
num_samples_GM_N = len(data_FA_N)
num_samples_RS = len(data_FA)
num_samples_RS_N = len(data_FA_N)


data_FA_combined = np.zeros((num_samples_FA + num_samples_FA_N, 76, 76))  
data_FA_combined[:num_samples_FA, :, :] = data_FA
data_FA_combined[num_samples_FA:, :, :] = data_FA_N


data_GM_combined = np.zeros((num_samples_GM + num_samples_GM_N, 76, 76))  
data_GM_combined[:num_samples_GM, :, :] = data_GM
data_GM_combined[num_samples_GM:, :, :] = data_GM_N


data_RS_combined = np.zeros((num_samples_RS + num_samples_RS_N, 76, 76))  
data_RS_combined[:num_samples_RS, :, :] = data_RS
data_RS_combined[num_samples_RS:, :, :] = data_RS_N

In [10]:
demographics_N['mstype'] = demographics_N['mstype'].apply(lambda x: 0 if x == -1 else 1)

### CARREGAR GRAFS

In [11]:
import networkx as nx
from networkx.algorithms.assortativity import average_degree_connectivity

In [12]:
def calculate_threshold(matrix):
    weights = matrix.values.flatten()
    mean_weight = weights.mean()
    std_weight = weights.std()
    median_weight = np.median(weights)
    threshold = mean_weight + std_weight  # media + 2 * std # mirar-ho amb la median i tornar mirar la FA
    return threshold

def create_graph_with_threshold(adj_matrix, nodes, threshold):
    G = nx.Graph()
    for index, row in nodes.iterrows():
        G.add_node(row['region_id'], name=row['region_name'])
    for i in range(adj_matrix.shape[0]):
        for j in range(i + 1, adj_matrix.shape[1]):
            if adj_matrix.iloc[i, j] > threshold:  # mayor q threshold
                G.add_edge(i+1, j+1, weight=adj_matrix.iloc[i, j])
    return G

#### Extreure mètriques de cada graf:

    Node: degree, strength, 3 centralitats
    Graf: average degree, average clustering and density
   

In [13]:
degree_fa = np.zeros(shape=(270,76), dtype=float)
strength_fa = np.zeros(shape=(270,76), dtype=float)
closeness_fa = np.zeros(shape=(270,76), dtype=float)
betweenness_fa = np.zeros(shape=(270,76), dtype=float)
eigenvector_fa = np.zeros(shape=(270,76), dtype=float)

average_degree_FA = np.zeros(shape=(270,1), dtype=float)
average_clustering_FA = np.zeros(shape=(270,1), dtype=float)
density_FA = np.zeros(shape=(270,1), dtype=float)

for i in range(data_FA_combined.shape[0]):
    print(f"Loading data_FA_combined[{i}]")
    adj_matrix = pd.DataFrame(data_FA_combined[i])
    threshold = calculate_threshold(adj_matrix)
    print(f"   Threshold = {threshold:.4f}")
    graph_FA = create_graph_with_threshold(adj_matrix, nodes, threshold)
    print(f"Grafo creado para muestra {i}: {graph_FA.number_of_nodes()} nodos, {graph_FA.number_of_edges()} aristas")

    # degree
    degrees = [val for (node, val) in graph_FA.degree()]
    degree_fa[i,:] = degrees

    # strength
    strengths = [val for (node, val) in graph_FA.degree(weight='weight')]
    strength_fa[i,:] = strengths

    # closeness
    closeness = nx.closeness_centrality(graph_FA)
    closeness_fa[i,:] = [closeness[node] for node in graph_FA.nodes()]

    # betweenness
    betweenness = nx.betweenness_centrality(graph_FA)
    betweenness_fa[i,:] = [betweenness[node] for node in graph_FA.nodes()]

    # eigenvector
    eigenvector = nx.eigenvector_centrality(graph_FA, max_iter=1000)
    eigenvector_fa[i,:] = [eigenvector[node] for node in graph_FA.nodes()]

    # average degree
    average_degree_FA[i, 0] = np.mean(degrees)

    # average clustering
    average_clustering_FA[i, 0] = nx.average_clustering(graph_FA)

    # density
    density_FA[i, 0] = nx.density(graph_FA)


df_degree_fa = pd.DataFrame(degree_fa)
df_degree_fa.to_csv('degree_fa.csv', index=False)

df_strength_fa = pd.DataFrame(strength_fa)
df_strength_fa.to_csv('strength_fa.csv', index=False)

df_closeness_fa = pd.DataFrame(closeness_fa)
df_closeness_fa.to_csv('closeness_fa.csv', index=False)

df_betweenness_fa = pd.DataFrame(betweenness_fa)
df_betweenness_fa.to_csv('betweenness_fa.csv', index=False)

df_eigenvector_fa = pd.DataFrame(eigenvector_fa)
df_eigenvector_fa.to_csv('eigenvector_fa.csv', index=False)

df_average_degree_FA = pd.DataFrame(average_degree_FA)
df_average_degree_FA.to_csv('average_degree_FA.csv', index=False)

df_average_clustering_FA = pd.DataFrame(average_clustering_FA)
df_average_clustering_FA.to_csv('average_clustering_FA.csv', index=False)

df_density_FA = pd.DataFrame(density_FA)
df_density_FA.to_csv('density_FA.csv', index=False)

print("Degree FA: {:.4f} - {:.4f}".format(np.min(degree_fa), np.max(degree_fa)))
print("Strength FA: {:.4f} - {:.4f}".format(np.min(strength_fa), np.max(strength_fa)))
print("Closeness FA: {:.4f} - {:.4f}".format(np.min(closeness_fa), np.max(closeness_fa)))
print("Betweenness FA: {:.4f} - {:.4f}".format(np.min(betweenness_fa), np.max(betweenness_fa)))
print("Eigenvector FA: {:.4f} - {:.4f}".format(np.min(eigenvector_fa), np.max(eigenvector_fa)))

print("Average Degree FA: {:.4f} - {:.4f}".format(np.min(average_degree_FA), np.max(average_degree_FA)))
print("Average Clustering FA: {:.4f} - {:.4f}".format(np.min(average_clustering_FA), np.max(average_clustering_FA)))
print("Density FA: {:.4f} - {:.4f}".format(np.min(density_FA), np.max(density_FA)))

Loading data_FA_combined[0]
   Threshold = 0.4428
Grafo creado para muestra 0: 76 nodos, 505 aristas
Loading data_FA_combined[1]
   Threshold = 0.4126
Grafo creado para muestra 1: 76 nodos, 439 aristas
Loading data_FA_combined[2]
   Threshold = 0.4698
Grafo creado para muestra 2: 76 nodos, 497 aristas
Loading data_FA_combined[3]
   Threshold = 0.4459
Grafo creado para muestra 3: 76 nodos, 469 aristas
Loading data_FA_combined[4]
   Threshold = 0.4144
Grafo creado para muestra 4: 76 nodos, 476 aristas
Loading data_FA_combined[5]
   Threshold = 0.4652
Grafo creado para muestra 5: 76 nodos, 437 aristas
Loading data_FA_combined[6]
   Threshold = 0.4134
Grafo creado para muestra 6: 76 nodos, 504 aristas
Loading data_FA_combined[7]
   Threshold = 0.4109
Grafo creado para muestra 7: 76 nodos, 437 aristas
Loading data_FA_combined[8]
   Threshold = 0.4051
Grafo creado para muestra 8: 76 nodos, 444 aristas
Loading data_FA_combined[9]
   Threshold = 0.4399
Grafo creado para muestra 9: 76 nodos, 44

Grafo creado para muestra 81: 76 nodos, 443 aristas
Loading data_FA_combined[82]
   Threshold = 0.4786
Grafo creado para muestra 82: 76 nodos, 442 aristas
Loading data_FA_combined[83]
   Threshold = 0.4587
Grafo creado para muestra 83: 76 nodos, 469 aristas
Loading data_FA_combined[84]
   Threshold = 0.4690
Grafo creado para muestra 84: 76 nodos, 438 aristas
Loading data_FA_combined[85]
   Threshold = 0.4657
Grafo creado para muestra 85: 76 nodos, 517 aristas
Loading data_FA_combined[86]
   Threshold = 0.4486
Grafo creado para muestra 86: 76 nodos, 460 aristas
Loading data_FA_combined[87]
   Threshold = 0.4856
Grafo creado para muestra 87: 76 nodos, 461 aristas
Loading data_FA_combined[88]
   Threshold = 0.4881
Grafo creado para muestra 88: 76 nodos, 438 aristas
Loading data_FA_combined[89]
   Threshold = 0.4466
Grafo creado para muestra 89: 76 nodos, 444 aristas
Loading data_FA_combined[90]
   Threshold = 0.4497
Grafo creado para muestra 90: 76 nodos, 501 aristas
Loading data_FA_combi

Grafo creado para muestra 160: 76 nodos, 511 aristas
Loading data_FA_combined[161]
   Threshold = 0.4809
Grafo creado para muestra 161: 76 nodos, 440 aristas
Loading data_FA_combined[162]
   Threshold = 0.4965
Grafo creado para muestra 162: 76 nodos, 481 aristas
Loading data_FA_combined[163]
   Threshold = 0.4507
Grafo creado para muestra 163: 76 nodos, 591 aristas
Loading data_FA_combined[164]
   Threshold = 0.4883
Grafo creado para muestra 164: 76 nodos, 513 aristas
Loading data_FA_combined[165]
   Threshold = 0.4740
Grafo creado para muestra 165: 76 nodos, 232 aristas
Loading data_FA_combined[166]
   Threshold = 0.4864
Grafo creado para muestra 166: 76 nodos, 268 aristas
Loading data_FA_combined[167]
   Threshold = 0.5013
Grafo creado para muestra 167: 76 nodos, 303 aristas
Loading data_FA_combined[168]
   Threshold = 0.4655
Grafo creado para muestra 168: 76 nodos, 214 aristas
Loading data_FA_combined[169]
   Threshold = 0.5215
Grafo creado para muestra 169: 76 nodos, 285 aristas
Lo

Loading data_FA_combined[240]
   Threshold = 0.5147
Grafo creado para muestra 240: 76 nodos, 266 aristas
Loading data_FA_combined[241]
   Threshold = 0.5340
Grafo creado para muestra 241: 76 nodos, 177 aristas
Loading data_FA_combined[242]
   Threshold = 0.5147
Grafo creado para muestra 242: 76 nodos, 295 aristas
Loading data_FA_combined[243]
   Threshold = 0.5117
Grafo creado para muestra 243: 76 nodos, 316 aristas
Loading data_FA_combined[244]
   Threshold = 0.5223
Grafo creado para muestra 244: 76 nodos, 263 aristas
Loading data_FA_combined[245]
   Threshold = 0.5229
Grafo creado para muestra 245: 76 nodos, 240 aristas
Loading data_FA_combined[246]
   Threshold = 0.5553
Grafo creado para muestra 246: 76 nodos, 215 aristas
Loading data_FA_combined[247]
   Threshold = 0.5210
Grafo creado para muestra 247: 76 nodos, 266 aristas
Loading data_FA_combined[248]
   Threshold = 0.5186
Grafo creado para muestra 248: 76 nodos, 223 aristas
Loading data_FA_combined[249]
   Threshold = 0.5047
Gra

In [14]:
degree_gm = np.zeros(shape=(270,76), dtype=float)
strength_gm = np.zeros(shape=(270,76), dtype=float)
closeness_gm = np.zeros(shape=(270,76), dtype=float)
betweenness_gm = np.zeros(shape=(270,76), dtype=float)
eigenvector_gm = np.zeros(shape=(270,76), dtype=float)

average_degree_GM = np.zeros(shape=(270,1), dtype=float)
average_clustering_GM = np.zeros(shape=(270,1), dtype=float)
density_GM = np.zeros(shape=(270,1), dtype=float)

for i in range(data_GM_combined.shape[0]):
    print(f"Loading data_GM_combined[{i}]")
    adj_matrix = pd.DataFrame(data_GM_combined[i])
    threshold = calculate_threshold(adj_matrix)
    print(f"   Threshold = {threshold:.4f}")
    graph_GM = create_graph_with_threshold(adj_matrix, nodes, threshold)
    print(f"Grafo creado para muestra {i}: {graph_GM.number_of_nodes()} nodos, {graph_GM.number_of_edges()} aristas")

    # degree
    degrees = [val for (node, val) in graph_GM.degree()]
    degree_gm[i,:] = degrees

    # strength
    strengths = [val for (node, val) in graph_GM.degree(weight='weight')]
    strength_gm[i,:] = strengths

    # closeness
    closeness = nx.closeness_centrality(graph_GM)
    closeness_gm[i,:] = [closeness[node] for node in graph_GM.nodes()]

    # betweenness
    betweenness = nx.betweenness_centrality(graph_GM)
    betweenness_gm[i,:] = [betweenness[node] for node in graph_GM.nodes()]

    # eigenvector
    eigenvector = nx.eigenvector_centrality(graph_GM, max_iter=1000)
    eigenvector_gm[i,:] = [eigenvector[node] for node in graph_GM.nodes()]

    # average degree
    average_degree_GM[i, 0] = np.mean(degrees)

    # average clustering
    average_clustering_GM[i, 0] = nx.average_clustering(graph_GM)

    # density
    density_GM[i, 0] = nx.density(graph_GM)


df_degree_gm = pd.DataFrame(degree_gm)
df_degree_gm.to_csv('degree_gm.csv', index=False)

df_strength_gm = pd.DataFrame(strength_gm)
df_strength_gm.to_csv('strength_gm.csv', index=False)

df_closeness_gm = pd.DataFrame(closeness_gm)
df_closeness_gm.to_csv('closeness_gm.csv', index=False)

df_betweenness_gm = pd.DataFrame(betweenness_gm)
df_betweenness_gm.to_csv('betweenness_gm.csv', index=False)

df_eigenvector_gm = pd.DataFrame(eigenvector_gm)
df_eigenvector_gm.to_csv('eigenvector_gm.csv', index=False)

df_average_degree_GM = pd.DataFrame(average_degree_GM)
df_average_degree_GM.to_csv('average_degree_GM.csv', index=False)

df_average_clustering_GM = pd.DataFrame(average_clustering_GM)
df_average_clustering_GM.to_csv('average_clustering_GM.csv', index=False)

df_density_GM = pd.DataFrame(density_GM)
df_density_GM.to_csv('density_GM.csv', index=False)

print("Degree GM: {:.4f} - {:.4f}".format(np.min(degree_gm), np.max(degree_gm)))
print("Strength GM: {:.4f} - {:.4f}".format(np.min(strength_gm), np.max(strength_gm)))
print("Closeness GM: {:.4f} - {:.4f}".format(np.min(closeness_gm), np.max(closeness_gm)))
print("Betweenness GM: {:.4f} - {:.4f}".format(np.min(betweenness_gm), np.max(betweenness_gm)))
print("Eigenvector GM: {:.4f} - {:.4f}".format(np.min(eigenvector_gm), np.max(eigenvector_gm)))

print("Average Degree GM: {:.4f} - {:.4f}".format(np.min(average_degree_GM), np.max(average_degree_GM)))
print("Average Clustering GM: {:.4f} - {:.4f}".format(np.min(average_clustering_GM), np.max(average_clustering_GM)))
print("Density GM: {:.4f} - {:.4f}".format(np.min(density_GM), np.max(density_GM)))

Loading data_GM_combined[0]
   Threshold = 0.1390
Grafo creado para muestra 0: 76 nodos, 310 aristas
Loading data_GM_combined[1]
   Threshold = 0.1418
Grafo creado para muestra 1: 76 nodos, 315 aristas
Loading data_GM_combined[2]
   Threshold = 0.1779
Grafo creado para muestra 2: 76 nodos, 331 aristas
Loading data_GM_combined[3]
   Threshold = 0.1624
Grafo creado para muestra 3: 76 nodos, 332 aristas
Loading data_GM_combined[4]
   Threshold = 0.1477
Grafo creado para muestra 4: 76 nodos, 319 aristas
Loading data_GM_combined[5]
   Threshold = 0.1489
Grafo creado para muestra 5: 76 nodos, 319 aristas
Loading data_GM_combined[6]
   Threshold = 0.1440
Grafo creado para muestra 6: 76 nodos, 321 aristas
Loading data_GM_combined[7]
   Threshold = 0.1464
Grafo creado para muestra 7: 76 nodos, 331 aristas
Loading data_GM_combined[8]
   Threshold = 0.1402
Grafo creado para muestra 8: 76 nodos, 325 aristas
Loading data_GM_combined[9]
   Threshold = 0.1527
Grafo creado para muestra 9: 76 nodos, 32

   Threshold = 0.1375
Grafo creado para muestra 80: 76 nodos, 313 aristas
Loading data_GM_combined[81]
   Threshold = 0.1678
Grafo creado para muestra 81: 76 nodos, 334 aristas
Loading data_GM_combined[82]
   Threshold = 0.1563
Grafo creado para muestra 82: 76 nodos, 324 aristas
Loading data_GM_combined[83]
   Threshold = 0.1845
Grafo creado para muestra 83: 76 nodos, 328 aristas
Loading data_GM_combined[84]
   Threshold = 0.1568
Grafo creado para muestra 84: 76 nodos, 332 aristas
Loading data_GM_combined[85]
   Threshold = 0.1241
Grafo creado para muestra 85: 76 nodos, 303 aristas
Loading data_GM_combined[86]
   Threshold = 0.1683
Grafo creado para muestra 86: 76 nodos, 329 aristas
Loading data_GM_combined[87]
   Threshold = 0.1587
Grafo creado para muestra 87: 76 nodos, 314 aristas
Loading data_GM_combined[88]
   Threshold = 0.1488
Grafo creado para muestra 88: 76 nodos, 327 aristas
Loading data_GM_combined[89]
   Threshold = 0.1660
Grafo creado para muestra 89: 76 nodos, 350 aristas

   Threshold = 0.1585
Grafo creado para muestra 159: 76 nodos, 329 aristas
Loading data_GM_combined[160]
   Threshold = 0.1526
Grafo creado para muestra 160: 76 nodos, 326 aristas
Loading data_GM_combined[161]
   Threshold = 0.1401
Grafo creado para muestra 161: 76 nodos, 328 aristas
Loading data_GM_combined[162]
   Threshold = 0.1460
Grafo creado para muestra 162: 76 nodos, 314 aristas
Loading data_GM_combined[163]
   Threshold = 0.1607
Grafo creado para muestra 163: 76 nodos, 336 aristas
Loading data_GM_combined[164]
   Threshold = 0.1499
Grafo creado para muestra 164: 76 nodos, 310 aristas
Loading data_GM_combined[165]
   Threshold = 0.6831
Grafo creado para muestra 165: 76 nodos, 11 aristas
Loading data_GM_combined[166]
   Threshold = 0.6736
Grafo creado para muestra 166: 76 nodos, 3 aristas
Loading data_GM_combined[167]
   Threshold = 0.6324
Grafo creado para muestra 167: 76 nodos, 352 aristas
Loading data_GM_combined[168]
   Threshold = 0.6231
Grafo creado para muestra 168: 76 no

Grafo creado para muestra 239: 76 nodos, 15 aristas
Loading data_GM_combined[240]
   Threshold = 0.6366
Grafo creado para muestra 240: 76 nodos, 161 aristas
Loading data_GM_combined[241]
   Threshold = 0.6883
Grafo creado para muestra 241: 76 nodos, 0 aristas
Loading data_GM_combined[242]
   Threshold = 0.6627
Grafo creado para muestra 242: 76 nodos, 2 aristas
Loading data_GM_combined[243]
   Threshold = 0.6775
Grafo creado para muestra 243: 76 nodos, 0 aristas
Loading data_GM_combined[244]
   Threshold = 0.6986
Grafo creado para muestra 244: 76 nodos, 2 aristas
Loading data_GM_combined[245]
   Threshold = 0.6561
Grafo creado para muestra 245: 76 nodos, 0 aristas
Loading data_GM_combined[246]
   Threshold = 0.6819
Grafo creado para muestra 246: 76 nodos, 0 aristas
Loading data_GM_combined[247]
   Threshold = 0.6693
Grafo creado para muestra 247: 76 nodos, 2 aristas
Loading data_GM_combined[248]
   Threshold = 0.6453
Grafo creado para muestra 248: 76 nodos, 12 aristas
Loading data_GM_co

In [15]:
degree_rs = np.zeros(shape=(270,76), dtype=float)
strength_rs = np.zeros(shape=(270,76), dtype=float)
closeness_rs = np.zeros(shape=(270,76), dtype=float)
betweenness_rs = np.zeros(shape=(270,76), dtype=float)
eigenvector_rs = np.zeros(shape=(270,76), dtype=float)

average_degree_RS = np.zeros(shape=(270,1), dtype=float)
average_clustering_RS = np.zeros(shape=(270,1), dtype=float)
density_RS = np.zeros(shape=(270,1), dtype=float)

for i in range(data_RS_combined.shape[0]):
    print(f"Loading data_RS_combined[{i}]")
    adj_matrix = pd.DataFrame(data_RS_combined[i])
    threshold = calculate_threshold(adj_matrix)
    print(f"   Threshold = {threshold:.4f}")
    graph_RS = create_graph_with_threshold(adj_matrix, nodes, threshold)
    print(f"Grafo creado para muestra {i}: {graph_RS.number_of_nodes()} nodos, {graph_RS.number_of_edges()} aristas")

    # degree
    degrees = [val for (node, val) in graph_RS.degree()]
    degree_rs[i,:] = degrees

    # strength
    strengths = [val for (node, val) in graph_RS.degree(weight='weight')]
    strength_rs[i,:] = strengths

    # closeness
    closeness = nx.closeness_centrality(graph_RS)
    closeness_rs[i,:] = [closeness[node] for node in graph_RS.nodes()]

    # betweenness
    betweenness = nx.betweenness_centrality(graph_RS)
    betweenness_rs[i,:] = [betweenness[node] for node in graph_RS.nodes()]

    # eigenvector
    eigenvector = nx.eigenvector_centrality(graph_RS, max_iter=1000)
    eigenvector_rs[i,:] = [eigenvector[node] for node in graph_RS.nodes()]

    # average degree
    average_degree_RS[i, 0] = np.mean(degrees)

    # average clustering
    average_clustering_RS[i, 0] = nx.average_clustering(graph_RS)

    # density
    density_RS[i, 0] = nx.density(graph_RS)


df_degree_rs = pd.DataFrame(degree_rs)
df_degree_rs.to_csv('degree_rs.csv', index=False)

df_strength_rs = pd.DataFrame(strength_rs)
df_strength_rs.to_csv('strength_rs.csv', index=False)

df_closeness_rs = pd.DataFrame(closeness_rs)
df_closeness_rs.to_csv('closeness_rs.csv', index=False)

df_betweenness_rs = pd.DataFrame(betweenness_rs)
df_betweenness_rs.to_csv('betweenness_rs.csv', index=False)

df_eigenvector_rs = pd.DataFrame(eigenvector_rs)
df_eigenvector_rs.to_csv('eigenvector_rs.csv', index=False)

df_average_degree_RS = pd.DataFrame(average_degree_RS)
df_average_degree_RS.to_csv('average_degree_RS.csv', index=False)

df_average_clustering_RS = pd.DataFrame(average_clustering_RS)
df_average_clustering_RS.to_csv('average_clustering_RS.csv', index=False)

df_density_RS = pd.DataFrame(density_RS)
df_density_RS.to_csv('density_RS.csv', index=False)

print("Degree RS: {:.4f} - {:.4f}".format(np.min(degree_rs), np.max(degree_rs)))
print("Strength RS: {:.4f} - {:.4f}".format(np.min(strength_rs), np.max(strength_rs)))
print("Closeness RS: {:.4f} - {:.4f}".format(np.min(closeness_rs), np.max(closeness_rs)))
print("Betweenness RS: {:.4f} - {:.4f}".format(np.min(betweenness_rs), np.max(betweenness_rs)))
print("Eigenvector RS: {:.4f} - {:.4f}".format(np.min(eigenvector_rs), np.max(eigenvector_rs)))

print("Average Degree RS: {:.4f} - {:.4f}".format(np.min(average_degree_RS), np.max(average_degree_RS)))
print("Average Clustering RS: {:.4f} - {:.4f}".format(np.min(average_clustering_RS), np.max(average_clustering_RS)))
print("Density RS: {:.4f} - {:.4f}".format(np.min(density_RS), np.max(density_RS)))


Loading data_RS_combined[0]
   Threshold = 0.5150
Grafo creado para muestra 0: 76 nodos, 501 aristas
Loading data_RS_combined[1]
   Threshold = 0.4391
Grafo creado para muestra 1: 76 nodos, 495 aristas
Loading data_RS_combined[2]
   Threshold = 0.4281
Grafo creado para muestra 2: 76 nodos, 471 aristas
Loading data_RS_combined[3]
   Threshold = 0.5125
Grafo creado para muestra 3: 76 nodos, 530 aristas
Loading data_RS_combined[4]
   Threshold = 0.4063
Grafo creado para muestra 4: 76 nodos, 483 aristas
Loading data_RS_combined[5]
   Threshold = 0.3985
Grafo creado para muestra 5: 76 nodos, 449 aristas
Loading data_RS_combined[6]
   Threshold = 0.5568
Grafo creado para muestra 6: 76 nodos, 558 aristas
Loading data_RS_combined[7]
   Threshold = 0.5084
Grafo creado para muestra 7: 76 nodos, 504 aristas
Loading data_RS_combined[8]
   Threshold = 0.4038
Grafo creado para muestra 8: 76 nodos, 477 aristas
Loading data_RS_combined[9]
   Threshold = 0.4767
Grafo creado para muestra 9: 76 nodos, 48

Grafo creado para muestra 81: 76 nodos, 507 aristas
Loading data_RS_combined[82]
   Threshold = 0.4312
Grafo creado para muestra 82: 76 nodos, 452 aristas
Loading data_RS_combined[83]
   Threshold = 0.4530
Grafo creado para muestra 83: 76 nodos, 511 aristas
Loading data_RS_combined[84]
   Threshold = 0.4502
Grafo creado para muestra 84: 76 nodos, 492 aristas
Loading data_RS_combined[85]
   Threshold = 0.5590
Grafo creado para muestra 85: 76 nodos, 507 aristas
Loading data_RS_combined[86]
   Threshold = 0.5565
Grafo creado para muestra 86: 76 nodos, 536 aristas
Loading data_RS_combined[87]
   Threshold = 0.4259
Grafo creado para muestra 87: 76 nodos, 446 aristas
Loading data_RS_combined[88]
   Threshold = 0.5665
Grafo creado para muestra 88: 76 nodos, 541 aristas
Loading data_RS_combined[89]
   Threshold = 0.5809
Grafo creado para muestra 89: 76 nodos, 535 aristas
Loading data_RS_combined[90]
   Threshold = 0.6913
Grafo creado para muestra 90: 76 nodos, 545 aristas
Loading data_RS_combi

   Threshold = 0.3786
Grafo creado para muestra 160: 76 nodos, 458 aristas
Loading data_RS_combined[161]
   Threshold = 0.4325
Grafo creado para muestra 161: 76 nodos, 477 aristas
Loading data_RS_combined[162]
   Threshold = 0.4630
Grafo creado para muestra 162: 76 nodos, 484 aristas
Loading data_RS_combined[163]
   Threshold = 0.5040
Grafo creado para muestra 163: 76 nodos, 539 aristas
Loading data_RS_combined[164]
   Threshold = 0.4296
Grafo creado para muestra 164: 76 nodos, 483 aristas
Loading data_RS_combined[165]
   Threshold = 0.3211
Grafo creado para muestra 165: 76 nodos, 355 aristas
Loading data_RS_combined[166]
   Threshold = 0.4059
Grafo creado para muestra 166: 76 nodos, 383 aristas
Loading data_RS_combined[167]
   Threshold = 0.5202
Grafo creado para muestra 167: 76 nodos, 415 aristas
Loading data_RS_combined[168]
   Threshold = 0.4138
Grafo creado para muestra 168: 76 nodos, 402 aristas
Loading data_RS_combined[169]
   Threshold = 0.4123
Grafo creado para muestra 169: 76

Grafo creado para muestra 239: 76 nodos, 380 aristas
Loading data_RS_combined[240]
   Threshold = 0.4830
Grafo creado para muestra 240: 76 nodos, 449 aristas
Loading data_RS_combined[241]
   Threshold = 0.3806
Grafo creado para muestra 241: 76 nodos, 374 aristas
Loading data_RS_combined[242]
   Threshold = 0.4320
Grafo creado para muestra 242: 76 nodos, 370 aristas
Loading data_RS_combined[243]
   Threshold = 0.4527
Grafo creado para muestra 243: 76 nodos, 421 aristas
Loading data_RS_combined[244]
   Threshold = 0.4206
Grafo creado para muestra 244: 76 nodos, 417 aristas
Loading data_RS_combined[245]
   Threshold = 0.4166
Grafo creado para muestra 245: 76 nodos, 425 aristas
Loading data_RS_combined[246]
   Threshold = 0.5021
Grafo creado para muestra 246: 76 nodos, 399 aristas
Loading data_RS_combined[247]
   Threshold = 0.3724
Grafo creado para muestra 247: 76 nodos, 369 aristas
Loading data_RS_combined[248]
   Threshold = 0.4705
Grafo creado para muestra 248: 76 nodos, 445 aristas
Lo